In [ ]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Data Import

The training data will have a real-values semantic textual relatedness score (between 0 and 1) for a pair of Marathi-language sentences.

The data is structured as a CSV file with the following fields:
- PairID: a unique identifier for the sentence pair
- Text: two sentences separated by a newline ('\n') character
- Score: the semantic textual relatedness score for the two sentences

Below we will show you how to load and re-format the provided data file.

In [ ]:
# Load the File
df_str_rel = pd.read_csv('https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/mar/mar_train.csv')
df_str_rel.head()

,PairID,Text,Score
0,MAR-train-00001,"""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता ...",1.00
1,MAR-train-00002,"""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन्...",1.00
2,MAR-train-00003,"""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमलात...",0.98
3,MAR-train-00004,"""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प्...",0.97
4,MAR-train-00005,"""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञानाच...",0.97


In [ ]:
y = df_str_rel['Score']

In [ ]:
# Creating a column "Split_Text" which is a list of two sentences.
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel.head()

,PairID,Text,Score,Split_Text
0,MAR-train-00001,"""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता ...",1.00,"[""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता..."
1,MAR-train-00002,"""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन्...",1.00,"[""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन..."
2,MAR-train-00003,"""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमलात...",0.98,"[""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमला..."
3,MAR-train-00004,"""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प्...",0.97,"[""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प..."
4,MAR-train-00005,"""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञानाच...",0.97,"[""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञाना..."


In [ ]:
# from tensorflow.keras.preprocessing.text import Tokenizer

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(df_str_rel['Split_Text'].apply(lambda x: x[0] + ' ' + x[1]))

# Convert each text to sequences
X1 = df_str_rel['Split_Text'].apply(lambda x: x[0])
X2 = df_str_rel['Split_Text'].apply(lambda x: x[1])

type(X1)

pandas.core.series.Series

In [ ]:
X1 = X1.values
X2 = X2.values
type(X1)

numpy.ndarray

In [ ]:
df_str_rel = df_str_rel.assign(sentence1 = X1)
df_str_rel = df_str_rel.assign(sentence2 = X2)
df_str_rel

,PairID,Text,Score,Split_Text,sentence1,sentence2
0,MAR-train-00001,"""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता ...",1.00,"[""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता...","""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता ...","""गुरु नानक देव आणि त्यांची शिकवण-दिक्षा संपूर्..."
1,MAR-train-00002,"""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन्...",1.00,"[""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन...","""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन्...","""2013च्या तुलनेत, आंतरराष्ट्रीय संस्था आणि एजन..."
2,MAR-train-00003,"""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमलात...",0.98,"[""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमला...","""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमलात...","""केंद्र सरकारच्या दृष्टीकोनानुसार हरियाणात योज..."
3,MAR-train-00004,"""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प्...",0.97,"[""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प...","""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प्...","""· सुरक्षित, सन्माननीय आणिशाश्वत पर्यटनाला प्र..."
4,MAR-train-00005,"""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञानाच...",0.97,"[""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञाना...","""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञानाच...","""या उदाहरणांवरून तंत्रज्ञानामुळे प्रशासनात सुध..."
...,...,...,...,...,...,...
1195,MAR-train-01196,"""लॉकडाऊनच्या काळात सर्वांचे मनोरंजन करणारा हा ...",0.03,"[""लॉकडाऊनच्या काळात सर्वांचे मनोरंजन करणारा हा...","""लॉकडाऊनच्या काळात सर्वांचे मनोरंजन करणारा हा ...","""त्यामुळे आता भाविकांनाही 500 आणि 1000च्या नोट..."
1196,MAR-train-01197,"""या वर्षी धोनीने 6 डावांमध्ये 150.50च्या सरासर...",0.03,"[""या वर्षी धोनीने 6 डावांमध्ये 150.50च्या सरास...","""या वर्षी धोनीने 6 डावांमध्ये 150.50च्या सरासर...","""इंडियन ग्रॉँप्रीमध्ये सलग तिस-या वर्षी मुख्य ..."
1197,MAR-train-01198,"""त्यासोबत भरतपूर जिल्ह्यातील कोरोना आकडेवारी ल...",0.03,"[""त्यासोबत भरतपूर जिल्ह्यातील कोरोना आकडेवारी ...","""त्यासोबत भरतपूर जिल्ह्यातील कोरोना आकडेवारी ल...","""ऑस्ट्रेलियन क्रिकेट प्रसारणाच्या ८०हून अधिक व..."
1198,MAR-train-01199,"""याप्रकरणी वाकड पोलीस ठाण्यात अज्ञात व्यक्तीवर...",0.03,"[""याप्रकरणी वाकड पोलीस ठाण्यात अज्ञात व्यक्तीव...","""याप्रकरणी वाकड पोलीस ठाण्यात अज्ञात व्यक्तीवर...","""रणबीर आणि आलिया सध्या बुल्गारियामध्ये त्यांच्..."


In [ ]:
df_str_rel.drop(columns = ['Text', 'Split_Text'], inplace = True)
df_str_rel

,PairID,Score,sentence1,sentence2
0,MAR-train-00001,1.00,"""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता ...","""गुरु नानक देव आणि त्यांची शिकवण-दिक्षा संपूर्..."
1,MAR-train-00002,1.00,"""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन्...","""2013च्या तुलनेत, आंतरराष्ट्रीय संस्था आणि एजन..."
2,MAR-train-00003,0.98,"""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमलात...","""केंद्र सरकारच्या दृष्टीकोनानुसार हरियाणात योज..."
3,MAR-train-00004,0.97,"""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प्...","""· सुरक्षित, सन्माननीय आणिशाश्वत पर्यटनाला प्र..."
4,MAR-train-00005,0.97,"""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञानाच...","""या उदाहरणांवरून तंत्रज्ञानामुळे प्रशासनात सुध..."
...,...,...,...,...
1195,MAR-train-01196,0.03,"""लॉकडाऊनच्या काळात सर्वांचे मनोरंजन करणारा हा ...","""त्यामुळे आता भाविकांनाही 500 आणि 1000च्या नोट..."
1196,MAR-train-01197,0.03,"""या वर्षी धोनीने 6 डावांमध्ये 150.50च्या सरासर...","""इंडियन ग्रॉँप्रीमध्ये सलग तिस-या वर्षी मुख्य ..."
1197,MAR-train-01198,0.03,"""त्यासोबत भरतपूर जिल्ह्यातील कोरोना आकडेवारी ल...","""ऑस्ट्रेलियन क्रिकेट प्रसारणाच्या ८०हून अधिक व..."
1198,MAR-train-01199,0.03,"""याप्रकरणी वाकड पोलीस ठाण्यात अज्ञात व्यक्तीवर...","""रणबीर आणि आलिया सध्या बुल्गारियामध्ये त्यांच्..."


In [ ]:
df_str_rel = df_str_rel[['PairID', 'sentence1', 'sentence2', 'Score']]
df_str_rel

,PairID,sentence1,sentence2,Score
0,MAR-train-00001,"""गुरु नानक देव, त्याचे शिकवण आणि आशीर्वाद आता ...","""गुरु नानक देव आणि त्यांची शिकवण-दिक्षा संपूर्...",1.00
1,MAR-train-00002,"""२०१ च्या तुलनेत आंतरराष्ट्रीय संस्था आणि एजन्...","""2013च्या तुलनेत, आंतरराष्ट्रीय संस्था आणि एजन...",1.00
2,MAR-train-00003,"""केंद्र सरकारच्या दृष्टीने यशस्वीरित्या अंमलात...","""केंद्र सरकारच्या दृष्टीकोनानुसार हरियाणात योज...",0.98
3,MAR-train-00004,"""g) सुरक्षित, सन्माननीय आणि टिकाऊ पर्यटनाला प्...","""· सुरक्षित, सन्माननीय आणिशाश्वत पर्यटनाला प्र...",0.97
4,MAR-train-00005,"""ही उदाहरणे प्रशासन सुधारण्यासाठी तंत्रज्ञानाच...","""या उदाहरणांवरून तंत्रज्ञानामुळे प्रशासनात सुध...",0.97
...,...,...,...,...
1195,MAR-train-01196,"""लॉकडाऊनच्या काळात सर्वांचे मनोरंजन करणारा हा ...","""त्यामुळे आता भाविकांनाही 500 आणि 1000च्या नोट...",0.03
1196,MAR-train-01197,"""या वर्षी धोनीने 6 डावांमध्ये 150.50च्या सरासर...","""इंडियन ग्रॉँप्रीमध्ये सलग तिस-या वर्षी मुख्य ...",0.03
1197,MAR-train-01198,"""त्यासोबत भरतपूर जिल्ह्यातील कोरोना आकडेवारी ल...","""ऑस्ट्रेलियन क्रिकेट प्रसारणाच्या ८०हून अधिक व...",0.03
1198,MAR-train-01199,"""याप्रकरणी वाकड पोलीस ठाण्यात अज्ञात व्यक्तीवर...","""रणबीर आणि आलिया सध्या बुल्गारियामध्ये त्यांच्...",0.03


# Preprocessing is done till here.

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e8cc0adaa472416c8a299615ac2c4c4bbe165e86577d11ef4147a882335d05c3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample
from sentence_transformers import losses

model = SentenceTransformer( "l3cube-pune/marathi-sentence-bert-nli" )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
ds = df_str_rel[ [ 'sentence1', 'sentence2', 'Score'] ].to_numpy()

train_examples = []
for i in range( ds.shape[0] ):
  train_examples.append( InputExample(texts=[ ds[i][0] , ds[i][1] ] , label=ds[i][2] ) )


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/75 [00:00<?, ?it/s]

# Training completed here

In [ ]:
test_ds = pd.read_csv( "https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/mar/mar_dev.csv" )
test_ds

,PairID,Text
0,MAR-dev-00001,"""या प्रसंगी जाहीर सभेला संबोधित करताना ते म्हण..."
1,MAR-dev-00002,"""गौतम गंभीरची संपती 147 कोटी रुपये असल्याची मा..."
2,MAR-dev-00003,"""खुद्द शरद पवार यांनी याबाबत स्थानिक नेत्यांचा..."
3,MAR-dev-00004,"""पेट्रोकेमिकल प्रकल्प आणून येथील फळबागायती आम्..."
4,MAR-dev-00005,"""त्यामुळे दक्षिण आफ्रिकेकडून सर्वाधिक बळी मिळव..."
...,...,...
288,MAR-dev-00289,"""पंतप्रधानांनी “कथन” म्हणून जे वर्णन केले ते भ..."
289,MAR-dev-00290,"""त्यावर आता राज्याच्या विधान परिषदेचे विरोधी प..."
290,MAR-dev-00291,"""या योजनेचे उद्दीष्ट सर्व घरांना शक्ती प्रदान ..."
291,MAR-dev-00292,"""शिवसेना व भाजपमध्ये युती झाल्यानंतर दोन्ही पक..."


In [ ]:
test_ds['Split_Text'] = test_ds['Text'].apply(lambda x: x.split("\n"))
test_ds.head()

,PairID,Text,Split_Text
0,MAR-dev-00001,"""या प्रसंगी जाहीर सभेला संबोधित करताना ते म्हण...","[""या प्रसंगी जाहीर सभेला संबोधित करताना ते म्ह..."
1,MAR-dev-00002,"""गौतम गंभीरची संपती 147 कोटी रुपये असल्याची मा...","[""गौतम गंभीरची संपती 147 कोटी रुपये असल्याची म..."
2,MAR-dev-00003,"""खुद्द शरद पवार यांनी याबाबत स्थानिक नेत्यांचा...","[""खुद्द शरद पवार यांनी याबाबत स्थानिक नेत्यांच..."
3,MAR-dev-00004,"""पेट्रोकेमिकल प्रकल्प आणून येथील फळबागायती आम्...","[""पेट्रोकेमिकल प्रकल्प आणून येथील फळबागायती आम..."
4,MAR-dev-00005,"""त्यामुळे दक्षिण आफ्रिकेकडून सर्वाधिक बळी मिळव...","[""त्यामुळे दक्षिण आफ्रिकेकडून सर्वाधिक बळी मिळ..."


In [ ]:
X1_dev = test_ds['Split_Text'].apply(lambda x: x[0])
X2_dev = test_ds['Split_Text'].apply(lambda x: x[1])

test_ds = test_ds.assign(sentence1 =X1_dev)
test_ds = test_ds.assign(sentence2 =X2_dev)

test_ds.drop(columns = ['Text', 'Split_Text'], inplace = True)

test_ds

,PairID,sentence1,sentence2
0,MAR-dev-00001,"""या प्रसंगी जाहीर सभेला संबोधित करताना ते म्हण...","""आज 500 कोटी रुपयांच्या नऊ उपक्रमांचे आज लोकार..."
1,MAR-dev-00002,"""गौतम गंभीरची संपती 147 कोटी रुपये असल्याची मा...","""गौतम गंभीर भाजपच्या तिकीटावर पूर्व दिल्ली लोक..."
2,MAR-dev-00003,"""खुद्द शरद पवार यांनी याबाबत स्थानिक नेत्यांचा...","""यात जिल्ह्यातील राजकीय समीकरणावर काय परिणाम ह..."
3,MAR-dev-00004,"""पेट्रोकेमिकल प्रकल्प आणून येथील फळबागायती आम्...","""कोकण उद्ध्वस्त होऊ नये म्हणून आमचा नाणार प्रक..."
4,MAR-dev-00005,"""त्यामुळे दक्षिण आफ्रिकेकडून सर्वाधिक बळी मिळव...","""एकंदरीत जबरदस्त परफॉर्मन्सने नटलेला हा ग्रँड ..."
...,...,...,...
288,MAR-dev-00289,"""पंतप्रधानांनी “कथन” म्हणून जे वर्णन केले ते भ...","""काही गोष्टी भारतात करणे केवळ अशक्य आहे अशा प्..."
289,MAR-dev-00290,"""त्यावर आता राज्याच्या विधान परिषदेचे विरोधी प...","""राजकारण हे कधीच चंचल नसतं, राजकीय नेते हे चंच..."
290,MAR-dev-00291,"""या योजनेचे उद्दीष्ट सर्व घरांना शक्ती प्रदान ...","""सर्व घरांमध्ये वीज पोहोचवणे हे या योजनेचे लक्..."
291,MAR-dev-00292,"""शिवसेना व भाजपमध्ये युती झाल्यानंतर दोन्ही पक...","""त्यामुळे ही नाराजी दूर करण्यासाठी मुख्यमंत्री..."


In [ ]:
def cosine( e1 , e2 ):
  return np.dot( e1.T , e2 ) / ( np.linalg.norm( e1 , ord=2 ) * np.linalg.norm( e2 , ord=2 ) )

test_ds = test_ds.to_numpy()
scores = []
for id , s1 , s2 in test_ds:
  e = model.encode( [ s1 , s2 ] )
  scores.append( cosine( e[0] , e[1] ) )


# Generate file for submission

Submission file has two columns: '**PairID**' and '**Pred_Score**'

In [ ]:
pred_ds = {
    "PairID": test_ds[ : , 0 ] ,
    "Pred_Score": scores
}
pred_ds = pd.DataFrame.from_dict( pred_ds )
pred_ds

,PairID,Pred_Score
0,MAR-dev-00001,0.833774
1,MAR-dev-00002,0.513987
2,MAR-dev-00003,0.768497
3,MAR-dev-00004,0.590228
4,MAR-dev-00005,0.330332
...,...,...
288,MAR-dev-00289,0.796085
289,MAR-dev-00290,0.297754
290,MAR-dev-00291,0.873563
291,MAR-dev-00292,0.669629


In [ ]:
pred_ds.to_csv( "pred_mar.csv", index = False )